In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import torch.nn as nn
import torch

In [3]:
# DATA를 저장하기 위한 리스트
vector_set =[]

# 파일을 열어서 vector_set에 저장한다
with open('/content/drive/MyDrive/IntroAI/plots.txt') as f:
    for line in f:
      vector_set.append(line.strip().split('\t'))
num_points = len(vector_set) 
print(num_points)


# numpy array 로 변환
# (x1, x2) (y)
x_data = np.array([ [ float(v[0]), float(v[1])] for v in vector_set], dtype=np.float32)
y_data = np.array([ [float(v[2])] for v in vector_set], dtype=np.float32)

x_data = x_data.reshape(-1, 2)
y_data = y_data.reshape(-1, 1)


1000


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
inputDim = 2        # takes variable 'x1, x2' 
outputDim = 1       # takes variable 'y'

# Network
class LinearRegression(nn.Module):

    def __init__(self, inputSize, outputSize):
        super().__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)

    def forward(self, x):
        out = self.linear(x)
        return out

model = LinearRegression(inputDim, outputDim).to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
loss_fn = torch.nn.MSELoss()   # Mean Squared Loss

In [6]:
epochs = 1000

for epoch in range(epochs):

    x = torch.from_numpy(x_data).to(device)
    y = torch.from_numpy(y_data).to(device)

    # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
    optimizer.zero_grad()

    # get output from the model, given the inputs
    out = model(x)

    # get loss for the predicted output
    loss = loss_fn(out, y)

    # get gradients w.r.t to parameters
    loss.backward()

    # update parameters
    optimizer.step()

    if epoch % 100 == 0:
      print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 0, loss 0.5588743090629578
epoch 100, loss 0.07999538630247116
epoch 200, loss 0.021660547703504562
epoch 300, loss 0.006725754123181105
epoch 400, loss 0.00253495667129755
epoch 500, loss 0.0013468266697600484
epoch 600, loss 0.0010092885931953788
epoch 700, loss 0.000913342519197613
epoch 800, loss 0.0008860630332492292
epoch 900, loss 0.000878307328093797


In [7]:
for p in model.parameters(): 
  print(p)


Parameter containing:
tensor([[ 0.3969, -0.3003]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.1707], device='cuda:0', requires_grad=True)


In [8]:
with torch.no_grad(): # we don't need gradients in the testing phase

    x = torch.from_numpy(x_data[:10]).to(device)
    y = torch.from_numpy(y_data[:10]).to(device)

    predicted = model(x).cpu().data.numpy()
    
    print('   정답--------예측')
    for (correct, predicted) in zip(y_data[:10], predicted):
      print(correct, predicted)


   정답--------예측
[0.5989284] [0.60353535]
[0.16104543] [0.17317905]
[0.3283041] [0.32828733]
[-0.5007947] [-0.4844051]
[-0.5137347] [-0.52264065]
[0.02729241] [0.09055763]
[-0.20024358] [-0.22829737]
[0.36755034] [0.3201686]
[0.27576628] [0.3189692]
[0.09480342] [0.08162116]
